In [ ]:
import pymysql
import pandas as pd
MIMICHOST = "35.233.174.193"
import ipywidgets as ipw
from IPython.display import display, HTML, clear_output
from collections import Counter
def get_mimic_connection():
    conn = pymysql.connect(
        host=MIMICHOST, port=3306, user="jovyan", passwd="jovyan", db="mimic2"
    )
    return conn


In [ ]:
conn = get_mimic_connection()

In [ ]:
rad_text = pd.read_sql("""SELECT hadm_id, icustay_id, charttime, text 
                          FROM noteevents
                          WHERE category='RADIOLOGY_REPORT'""",
                       conn)
rad_text

In [ ]:
@ipw.interact(df=ipw.fixed(rad_text), i=ipw.IntSlider(max=rad_text.shape[0]))
def view_report(df,i):
    #clear_output()
    print((df.loc[i,"text"]))

In [ ]:
import spacy

In [ ]:
import re

In [ ]:
#!python -m pip install https://github.com/abchapman93/spacy_models/raw/master/releases/en_info_3700_i2b2_2012-0.1.0/dist/en_info_3700_i2b2_2012-0.1.0.tar.gz

In [ ]:
nlp = spacy.load("en_info_3700_i2b2_2012")

In [ ]:
#from clinical_sectionizer import Sectionizer

In [ ]:
#s = Sectionizer(nlp)

In [ ]:
from spacy.matcher import Matcher
# Add match ID "HelloWorld" with no callback and one pattern


In [ ]:
with nlp.disable_pipes('tagger','parser', 'ner'):
    docs = list(nlp.pipe(rad_text.text))

In [ ]:
def prune_overlapping_matches(matches, strategy="longest"):
    if strategy != "longest":
        raise NotImplementedError()

    # Make a copy and sort
    unpruned = sorted(matches, key=lambda x: (x[1], x[2]))
    pruned = []
    num_matches = len(matches)
    if num_matches == 0:
        return matches
    curr_match = unpruned.pop(0)

    while True:
        if len(unpruned) == 0:
            pruned.append(curr_match)
            break
        next_match = unpruned.pop(0)

        # Check if they overlap
        if overlaps(curr_match, next_match):
            # Choose the larger span
            longer_span = max(curr_match, next_match, key=lambda x: (x[2] - x[1]))
            pruned.append(longer_span)
            if len(unpruned) == 0:
                break
            curr_match = unpruned.pop(0)
        else:
            pruned.append(curr_match)
            curr_match = next_match
    # Recursive base point
    if len(pruned) == num_matches:
        return pruned
    # Recursive function call
    else:
        return prune_overlapping_matches(pruned)

def overlaps(a, b):
    if _span_overlaps(a, b) or _span_overlaps(b, a):
        return True
    return False

def _span_overlaps(a, b):
    _, a_start, a_end = a
    _, b_start, b_end = b
    if a_start >= b_start and a_start < b_end:
        return True
    if a_end > b_start and a_end <= b_end:
        return True
    return False

def matches_to_spans(doc, matches, set_label=True):
    spans = []
    for (rule_id, start, end) in matches:
        if set_label:
            label = doc.vocab.strings[rule_id]
        else:
            label = None
        spans.append(Span(doc, start=start, end=end, label=label))
    return spans

In [ ]:
matcher = Matcher(nlp.vocab)
pattern1 = [{"TEXT": {"REGEX": "[A-Z\/-]+"}, "OP":"*"},
           {"IS_UPPER": True},
           {"TEXT":":"}]
matcher.add("POTENTIAL_SECTIONS", None, pattern1)


In [ ]:
r1 = re.compile(r"""[A-Z]{1,1}[A-Z\s/]*[A-Z]{1,1}:""")
txt = "of allergies. CHEST: W/CONTRAST: There is no axillary lymphadenopathy."
txt2= "blah blah blah INTRAVENOUS CONTRAST: and blah"
#print(r1.findall(txt))
#print(r1.findall(txt2))

In [ ]:
d = nlp(txt+txt2)

In [ ]:
for m in matcher(d):
    print(m)
    print(d[m[1]:m[2]].text)

In [ ]:
matches = matcher(d)
prune_overlapping_matches(matches)

In [ ]:
psections = [d[start:end].text.strip() for d in docs for match_id, start, end in prune_overlapping_matches(matcher(d))]

In [ ]:
Counter(psections).most_common(50)

In [ ]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

In [ ]:
spacy.__version__

In [ ]:
matcher2 = Matcher(nlp.vocab)
pattern2 = [{"lower": "addendum"},
           ]
matcher2.add("POTENTIAL_SECTIONS", None, pattern2)

In [ ]:
addendums = [i for i in range(len(docs)) if prune_overlapping_matches(matcher2(docs[i]))]

In [ ]:
len(addendums)

In [ ]:
addendums

In [ ]:
matcher3 = Matcher(nlp.vocab)
pattern3 = [{"TEXT": "IMPRESSION"}, {"TEXT":":"},]
matcher3.add("IMPRESS1", None, pattern3)
matcher3(nlp("IMPRESSION:"))

In [ ]:
no_impression = [i for i in range(len(docs)) if not bool(prune_overlapping_matches(matcher3(docs[i])))]

In [ ]:
len(no_impression)
    

In [ ]:
matcher3(docs[2])

In [ ]:
[prune_overlapping_matches(matcher3(docs[i])) for i in range(20)]

In [ ]:
@ipw.interact(df=ipw.fixed(rad_text.iloc[no_impression,:].reset_index()), i=ipw.IntSlider(max=rad_text.iloc[no_impression,:].shape[0]))
def _view_report(df,i):
    #clear_output()
    print((df.loc[i]))

In [ ]:
tmp = rad_text.iloc[no_impression,:]
tmp.shape

In [ ]:
tmp.reset_index()

In [ ]:
docs[no_impression[7279]], no_impression[7279]

In [ ]:
no_impression[7907]

In [ ]:
docs[no_impression[7907]]

In [ ]:
matcher = Matcher(nlp.vocab)
pattern1 = [{"TEXT": {"REGEX": "[A-Z\/-]+"}, "OP":"*"},
           {"IS_UPPER": True},
           {"TEXT":":", "OP":"?"}]
matcher.add("POTENTIAL_SECTIONS", None, pattern1)